In [12]:
from __future__ import print_function

import glob
import math
import os

from IPython import display
from matplotlib import cm
from matplotlib import gridspec
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn import metrics
import tensorflow as tf
from tensorflow.python.data import Dataset

tf.logging.set_verbosity(tf.logging.ERROR)
pd.options.display.max_rows = 10
pd.options.display.float_format = '{:.1f}'.format
INPUT_FEATURE = 'pixels'
SHAPE = 784
def parse_labels_and_features(dataset):
  """Extracts labels and features.
  
  This is a good place to scale or transform the features if needed.
  
  Args:
    dataset: A Pandas `Dataframe`, containing the label on the first column and
      monochrome pixel values on the remaining columns, in row major order.
  Returns:
    A `tuple` `(labels, features)`:
      labels: A Pandas `Series`.
      features: A Pandas `DataFrame`.
  """
  labels = dataset[0]

  # DataFrame.loc index ranges are inclusive at both ends.
  features = dataset.loc[:,1:784]
  # Scale the data to [0, 1] by dividing out the max value, 255.
  features = features / 255

  return labels, features

def construct_feature_columns():
  """Construct the TensorFlow Feature Columns.

  Returns:
    A set of feature columns
  """ 
  
  # There are 784 pixels in each image.
  return set([tf.feature_column.numeric_column(INPUT_FEATURE, shape=SHAPE)])

def create_predict_input_fn(features, labels, batch_size):
  """A custom input_fn for sending mnist data to the estimator for predictions.

  Args:
    features: The features to base predictions on.
    labels: The labels of the prediction examples.

  Returns:
    A function that returns features and labels for predictions.
  """
  def _input_fn():
    raw_features = {INPUT_FEATURE: features.values}
    raw_targets = np.array(labels)
    
    ds = Dataset.from_tensor_slices((raw_features, raw_targets)) # warning: 2GB limit
    ds = ds.batch(batch_size)
    
        
    # Return the next batch of data.
    feature_batch, label_batch = ds.make_one_shot_iterator().get_next()
    return feature_batch, label_batch

  return _input_fn

def inference(examples, targets, batch_size, hidden_units, model_dir):
    predict_training_input_fn = create_predict_input_fn(examples, targets, batch_size)
    classifier = tf.estimator.DNNClassifier(
      feature_columns=construct_feature_columns(),
      hidden_units=hidden_units,
      n_classes=10,
      config=tf.estimator.RunConfig(keep_checkpoint_max=1),
      model_dir=model_dir
    )
    predictions = list(classifier.predict(input_fn=predict_training_input_fn))
    class_id = np.array([item['class_ids'][0] for item in predictions])
    return class_id


In [13]:

mnist_test_dataframe = pd.read_csv(
  "mnist_test.csv",
  sep=",",
  header=None)

test_targets, test_examples = parse_labels_and_features(mnist_test_dataframe)
single_target = test_targets.head(1)
single_example = test_examples.head(1)
print(single_target)
print(single_example)

test_class = inference(examples=single_example, targets=single_target, batch_size=1, hidden_units=[128, 128, 128], model_dir='models/mnist.ckpt')
print("test prediction", test_class)

0    7
Name: 0, dtype: int64
   1    2    3    4    5    6    7    8    9    10  ...   775  776  777  778  \
0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0 ...   0.0  0.0  0.0  0.0   

   779  780  781  782  783  784  
0  0.0  0.0  0.0  0.0  0.0  0.0  

[1 rows x 784 columns]
test prediction [7]
